In [26]:
import os
import sqlite3
import pandas as pd
import json
import re  # Import the re module for regular expressions

# Path where files are located
files_directory = r'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3'

# Define participant labels to delete
participant_labels_to_delete = {'evgeny', 'jmgodber', 'csaxman', 'sas14', 'bouwman'}

# List to store files to be deleted
files_to_delete = []

# List to store files remaining after deletion
remaining_files = []

# Function to rename files by adding .sqlite extension
def add_sqlite_extension(file_path):
    if not file_path.endswith('.sqlite'):
        new_file_path = file_path + '.sqlite'
        os.rename(file_path, new_file_path)
        print(f"Renamed '{file_path}' to '{new_file_path}'")
        return new_file_path
    return file_path

# Function to safely extract JSON data
def safe_from_json(json_string):
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        print("Error decoding JSON.")
        return None

# Function to extract text from JSON
def extract_text_from_json(json_data):
    extracted_text = []
    def extract_text(obj):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == 'text':
                    extracted_text.append(v)
                extract_text(v)
        elif isinstance(obj, list):
            for item in obj:
                extract_text(item)
    extract_text(json_data)
    return '\n'.join(extracted_text)

# DataFrame to combine results
combined_df = pd.DataFrame()

# Process each file in the directory
for filename in os.listdir(files_directory):
    file_path = os.path.join(files_directory, filename)
    
    if os.path.isfile(file_path):
        # Rename the file by adding .sqlite extension if needed
        new_file_path = add_sqlite_extension(file_path)
        
        # Connect to the SQLite database
        try:
            conn = sqlite3.connect(new_file_path)
            c = conn.cursor()

            # Query the 'props' table to get the 'value' for 'metadata'
            metadata_query = "SELECT value FROM props WHERE name = 'metadata'"
            metadata_df = pd.read_sql_query(metadata_query, conn)
            
            # Check if 'metadata_df' has data
            if metadata_df.empty:
                print("No metadata found in 'props' table.")
                conn.close()
                continue
            
            # Extract the participant 'netid' from the 'metadata'
            metadata_value = metadata_df['value'][0]  # Assuming only one entry for metadata
            metadata = safe_from_json(metadata_value)
            if metadata is None:
                print("Error extracting participant from metadata.")
                conn.close()
                continue
            participant = metadata.get('participant', 'Unknown')
            
            # Query the 'document' table to get the 'final' JSON information
            document_query = "SELECT json FROM document WHERE kind = 'final'"
            document_df = pd.read_sql_query(document_query, conn)
            
            # Check if 'document_df' has data
            if document_df.empty:
                print("No final JSON information found in 'document' table.")
                conn.close()
                continue
            
            # Extract the 'final' JSON information
            final_json = document_df['json'][0]  # Assuming only one entry for final
            
            # Extract text using the function
            extracted_text = extract_text_from_json(safe_from_json(final_json))
            
            # Create a data frame with participant and extracted text
            temp_df = pd.DataFrame({
                'file_path': [new_file_path],
                'participant': [participant],
                'extracted_text': [extracted_text]
            })
            
            # Combine with the main DataFrame
            combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

            conn.close()
        
        except sqlite3.Error as e:
            print(f"Error accessing SQLite file {new_file_path}: {e}")
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file {new_file_path}")

# Retain only the file with the most text for each participant
if not combined_df.empty:
    combined_df['text_length'] = combined_df['extracted_text'].apply(len)
    
# Group by participant and find the index of the max text length
    max_text_length_df = combined_df.loc[combined_df.groupby('participant')['text_length'].idxmax()]
    
    # Create a set of file paths to keep
    remaining_files = set(max_text_length_df['file_path'])
    
    # Identify files to delete (all other files for each participant)
    for index, row in combined_df.iterrows():
        if row['file_path'] not in remaining_files or row['participant'] in participant_labels_to_delete:
            files_to_delete.append(row['file_path'])

# Remove duplicates from files_to_delete
files_to_delete = list(set(files_to_delete))

# Delete the identified files
for file_path in files_to_delete:
    try:
        os.remove(file_path)
        print(f"Deleted file: {file_path}")
    except IOError:
        print(f"Error deleting file: {file_path}")

# Perform additional SQL operations on remaining files
for filename in os.listdir(files_directory):
    file_path = os.path.join(files_directory, filename)
    
    if os.path.isfile(file_path):
        new_file_path = add_sqlite_extension(file_path)
        
        try:
            conn = sqlite3.connect(new_file_path)
            c = conn.cursor()

            # Update metadata to replace participant with an empty string
            c.execute("SELECT value FROM props WHERE name='metadata'")
            metadata_row = c.fetchone()
            if metadata_row:
                metadata_value = metadata_row[0]
                try:
                    metadata_json = json.loads(metadata_value)
                    metadata_json['participant'] = ""
                    new_metadata_value = json.dumps(metadata_json)
                    c.execute("UPDATE props SET value=? WHERE name='metadata'", (new_metadata_value,))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON for file: {filename}. Value: {metadata_value}")
        
            # Update config to replace label with an empty string (if applicable)
            c.execute("SELECT value FROM props WHERE name='config'")
            config_row = c.fetchone()
            if config_row:
                config_value = config_row[0]
                try:
                    config_json = json.loads(config_value)
                    config_json['label'] = ""
                    new_config_value = json.dumps(config_json)
                    c.execute("UPDATE props SET value=? WHERE name='config'", (new_config_value,))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON for file: {filename}. Value: {config_value}")

            # Clear tabs
            c.execute("UPDATE act SET json='{\"is_focused\":false}' WHERE k='focus' AND json LIKE '%is_focused\":false%'")

            conn.commit()
            conn.close()
        
        except sqlite3.Error as e:
            print(f"Error accessing SQLite file {new_file_path}: {e}")

# Rename log files after processing
for filename in os.listdir(files_directory):
    file_path = os.path.join(files_directory, filename)
    
    if os.path.isfile(file_path):
        current_name = file_path
        new_name = re.sub(r"(\d+)(-\w+)(-\w+)(.sqlite)", r"7399\2\3", filename)
        new_file_path = os.path.join(files_directory, new_name)
        try:
            os.rename(current_name, new_file_path)
            print(f"Renamed '{current_name}' to '{new_file_path}'")
        except Exception as e:
            print(f"Skipping {current_name} - unable to rename file ({e})")

print("All specified operations are complete.")


Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiymbu-0xp5jiBpdI' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiymbu-0xp5jiBpdI.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiyogy-V88kCZA8PX' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiyogy-V88kCZA8PX.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiyz5p-JWFtYxC3Iz' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eiyz5p-JWFtYxC3Iz.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eizo2a-pImzTCWsZI' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3eizo2a-pImzTCWsZI.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3ej12ip-r95wPzmA17' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3ej12ip-r95wPzmA17.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300

Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hh3ukx-2c8jjwnV85' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hh3ukx-2c8jjwnV85.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hhfffh-oQ6ihnHrpN' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hhfffh-oQ6ihnHrpN.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hhv6a1-IVyzFiZ1j6' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hhv6a1-IVyzFiZ1j6.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hk7te7-ZZrclzsdbP' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hk7te7-ZZrclzsdbP.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hk7tus-NbDFNleJHz' to 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300-m3hk7tus-NbDFNleJHz.sqlite'
Renamed 'C:\Users\Wren\Documents\SW Fall 2024\researchers\round 3\7300